In [1]:
import tensorflow as tf
import tflearn

In [2]:
import numpy as np

In [3]:
sess = tf.Session()
optim = tf.train.AdamOptimizer(learning_rate=0.0001)

In [4]:
from tqdm import tqdm

In [5]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [6]:
import BayesianNN

In [7]:
bnn = BayesianNN.BayesianFC([4,128,128,1])

In [8]:
def test_func(x):
    e = np.random.normal(0,0.01)
    return x + 0.3*np.sin(2 *np.pi * (x + e)) + 0.3*np.sin(4 * np.pi*(x + e)) + e

In [9]:
data = [(x, test_func(x)) for x in np.linspace(0,0.5,1000)]

In [10]:
data_input = list(map(lambda x: [x[0]], data))
data_input_poly = list(map(lambda x: [x[0], x[0]**2, x[0]**3, x[0]**4], data))
data_target = list(map(lambda x: [x[1]], data))

In [11]:
def normal_nn(optim):
    data_input = tf.placeholder(tf.float32, shape=[None, 1])
    data_target = tf.placeholder(tf.float32, shape=[None, 1])
    
    poly = tf.concat(1, [data_input, data_input**2, data_input**3, data_input**4])
    net = tflearn.fully_connected(poly, 128, activation="relu")
    net = tflearn.fully_connected(net, 128, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
    
    data_output = tflearn.fully_connected(net, 1, activation="linear")
    
    loss = tf.reduce_mean(tf.square(data_output - data_target))
    op = optim.minimize(loss)
    
    return op, data_input, data_target, loss, data_output

In [12]:
batch_size = 1000

In [13]:
epoch_loss = []
nn_epoch_loss = []
epochs = 1000

In [14]:
test_set = [[x] for x in np.linspace(-0.5,1,1000)]
test_set_poly = [[x,x**2,x**3,x**4] for x in np.linspace(-0.5,1,1000)]
test_set_flat = [a[0] for a in test_set]
data_input_flat = [a[0] for a in data_input]
data_target_flat = [a[0] for a in data_target]

In [15]:
op, data_x, data_y, loss, data_output = normal_nn(optim)
sess.run(tf.global_variables_initializer())
for e in tqdm(range(epochs)):
    input_batch = data_input
    target_batch = data_target

    _, l = sess.run([op, loss], feed_dict={data_x: input_batch, data_y:target_batch})
    nn_epoch_loss.append(l)

100%|██████████| 5000/5000 [00:17<00:00, 290.36it/s]


In [16]:
nn_preds = sess.run(data_output, feed_dict={data_x: test_set})

In [17]:
op, data_x, data_y, m_scaling, loss, mean_grads = bnn.update(8, optim, batch_size)
sess.run(tf.global_variables_initializer())

In [25]:
for e in tqdm(range(epochs)):
#     indices = np.random.randint(low=0, high=1000, size=100)
#     input_batch = [data_input[i] for i in indices]
#     target_batch = [data_target[i] for i in indices]
    input_batch = data_input_poly
    target_batch = data_target
    ms = (2**(epochs - e - 1))/( (2**epochs) - 1) 
    
#     grad_nums = [a for (a,b) in grads]
    _, l = sess.run([op, loss], feed_dict={data_x: input_batch, data_y:target_batch, m_scaling:[ms]})
    epoch_loss.append(l)
    
#     print()
#     print(wg)
#     print()
#     print(mg)
#     print()
#     print(sg)
#     print()
        

#     print("Epoch {}:".format(e))
#     print(sess.run([bnn.qb_mean, bnn.qb_p]))
#     print(g)
#     print()
    #     print("---")
    #     print(sess.run([bnn.qw_p, bnn.qb_p]))    

100%|██████████| 5000/5000 [02:39<00:00, 31.36it/s]


In [26]:
# sess.run(bnn.qw_ps[0])

In [27]:
pred_input, pred_output = bnn.sample()
predictions = sess.run(pred_output, feed_dict={pred_input: test_set_poly})
var_trials = 100
for _ in tqdm(range(var_trials-1)):
    predictions = np.concatenate([predictions, sess.run(pred_output, feed_dict={pred_input: test_set_poly})], axis=1)

100%|██████████| 99/99 [00:00<00:00, 595.95it/s]


In [28]:
pred_means = np.apply_along_axis(lambda x: np.mean(x), 1, predictions)
pred_means[3]

0.37224704

In [29]:
pred_vars = np.apply_along_axis(lambda x: np.std(x, ddof=1), 1, predictions)

In [30]:
p = figure(width=500, height=500)

err_xs=[]
err_ys=[]
for x, y, yerr in zip(test_set_flat, pred_means, pred_vars):
    err_xs.append((x, x))
    err_ys.append((y - yerr, y + yerr))

p.multi_line(err_xs, err_ys, color=(255,0,0,0.15))

# p.line([a/100 for a in range(100)], [scale*a/100 for a in range(100)], line_width=3, color="blue")
p.line(test_set_flat, pred_means, line_width=2, color="red")

p.line(data_input_flat, data_target_flat, line_width=3, color="blue")

p.line(test_set_flat, nn_preds.flatten(), line_width=2, color="orange")

# Loss
# p.line(range(epochs), epoch_loss, line_width=2, color="green")

show(p)

In [31]:
loss_p = figure(width=500, height=500)

# Loss
loss_p.line(range(len(epoch_loss)), epoch_loss, line_width=2, color="green")

loss_p.line(range(len(nn_epoch_loss)), nn_epoch_loss, line_width=2, color="purple")

show(loss_p)